In [492]:
import pandas as pd
import json

In [493]:
df = pd.read_csv('../csv/time_series_covid19_recovered_global.csv', index_col=0)
df = df.drop(columns=['Lat', 'Long'])


In [494]:
df = df.reset_index().set_index('Country/Region')
df = df.rename({'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Democratic Republic of Congo'})

In [495]:
def merge_provinces(country_name, df):
    country = df.loc[[country_name]].drop(columns=['Province/State'])
    country = country.sum(axis=0).rename(country_name)
    df = df.drop([country_name]).append(country)
    return df

In [496]:
# Dropping the ships
df = df.drop(['Diamond Princess'])
df = df.drop(['MS Zaandam'])
# Merge countries with provinces into a single row
df = merge_provinces('China', df)
df = merge_provinces('Australia', df)

In [497]:
mask = (df['Province/State'] != '0') & (pd.isnull(df['Province/State']) != True)
df2 = df.loc[mask]
df2 = df2.set_index('Province/State')
df2.index = df2.index.rename('Country/Region')

In [498]:
df.loc['China']

Province/State      NaN
1/22/20              28
1/23/20              30
1/24/20              36
1/25/20              39
                  ...  
3/31/20           76206
4/1/20            76405
4/2/20            76565
4/3/20            76760
4/4/20            76946
Name: China, Length: 75, dtype: object

In [499]:
df = df[(df['Province/State'] == '0') | (pd.isnull(df['Province/State']) == True)]

In [500]:
df = df.append(df2)
df = df.drop(columns='Province/State')

In [501]:
df = df.stack().reset_index(level=1)
df = df.rename(columns={'level_1': 'date', 0: 'recovered'})

In [502]:
df['date'] = pd.to_datetime(df['date']).apply(lambda x: x.strftime('%Y-%m-%d'))

In [503]:
df = df.rename({'Cabo Verde': 'Cape Verde', 'US': 'United States', 'West Bank and Gaza': 'Palestine', 'Burma': 'Myanmar', 'Channel Islands': 'Jersey', 'Holy See': 'Vatican', 'Korea, South': 'South Korea', 'Falkland Islands (Islas Malvinas)': 'Falkland Islands', 'Sint Maarten': 'Sint Maarten (Dutch part)', 'Taiwan*': 'Taiwan','St Martin' :'Saint Martin'})

In [504]:
full_data = pd.read_csv('../data/full_data.csv', index_col=0)

In [505]:
difference = df.index.difference(full_data.index).to_list()
full_diff = full_data.index.difference(df.index).to_list()

In [506]:
alpha3 = pd.read_csv('../data/country_codes.csv')
pattern = {'ç': 'c', 'ô': 'o', '’': "'", 'é': 'e'}
alpha3['Country or Area'] = alpha3['Country or Area'].replace(pattern, regex=True)
alpha3['Country or Area'] = alpha3['Country or Area'].str.replace('\(.*\)', '',regex=True)
alpha3['Country or Area'] = alpha3['Country or Area'].str.replace('\s+the', '',regex=True)
alpha3['Country or Area'] = alpha3['Country or Area'].str.strip()
alpha3 = alpha3.set_index('Country or Area')
alpha3 = alpha3['ISO-alpha3 code']

In [507]:
country_fixes = {'World': 'World', 'Brunei': 'BRN', 'Vatican': 'VAT', 'United States': 'USA', 'United Kingdom': 'GBR', 'International': 'International', 'Tanzania': 'TZA', 'Kosovo': 'RKS','Taiwan': 'TWN', 'Laos': 'LAO', 'Syria': 'SYR', 'Moldova': 'MDA', 'South Korea': 'KOR', 'Palestine': 'PSE', 'Russia': 'RUS', 'Sint Maarten (Dutch part)': 'SXM', 'Saint Vincent and the Grenadines': 'VCT'}
country_fixes = pd.Series(country_fixes)
alpha3 = alpha3.append(country_fixes)

In [508]:
df

,date,recovered
Country/Region,,
Afghanistan,2020-01-22,0.0
Afghanistan,2020-01-23,0.0
Afghanistan,2020-01-24,0.0
Afghanistan,2020-01-25,0.0
Afghanistan,2020-01-26,0.0
...,...,...
Falkland Islands,2020-03-31,0.0
Falkland Islands,2020-04-01,0.0
Falkland Islands,2020-04-02,0.0


In [509]:
df['Country'] = df.index.get_level_values(0)
df['alpha3'] = df['Country'].map(alpha3)
df = df.drop(columns='Country')

In [510]:
data = {}
for group in df.groupby(level=0):
    data[group[0]] = {
        'alpha3': group[1]['alpha3'].unique()[0],
        'recovered': group[1][['date', 'recovered']].values.tolist()
    }

In [511]:
file_name = 'recovery_data'
open(f'../json/{file_name}.json', 'w').write(json.dumps(data))

329127